<a href="https://colab.research.google.com/github/Francescolv99/Deep-Video-Deblurring/blob/main/Deep_Video_Deblurring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ottengo il dataset scaricandolo dal link in questione


In [ ]:
!wget http://www.cs.ubc.ca/labs/imager/tr/2017/DeepVideoDeblurring/DeepVideoDeblurring_Dataset.zip
!unzip /content/DeepVideoDeblurring_Dataset.zip

Importo i pacchetti necessari

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
import pandas as pd
import os

Creo le variabili x e y e le funzioni necessarie per lavorare il dataset

In [ ]:
#modificare le funzioni per prendere un dataset corretto e non troppo costoso


#dim immagini 720x1280x3
x = []
y = []

def appendImmagine(filename, x):
    im_1 = Image.open(filename)
    ar = np.array(im_1)
    x.append(ar)
    return x
    

#accedo ad ogni immagine nel database e le inserisco in forma matriciale in x e y
def ottieniImmagini(filename, x , y):
    #apro la cartella del dataset
    entries = os.listdir(filename)
    #entries.pop(0)
    #apro ogni sottocartella
    print("LOADING DATASET:")
    for cartella in entries:
        if(cartella==".DS_Store"):
              continue
        lista_cartella = os.listdir(filename + '/'+ cartella)
        #lista_cartella.pop(0)
        #lista_cartella è la lista di sottocartelle all'interno di "qualitative o quantitative dataset"
        for cartella2 in lista_cartella:
          #cartella2 sarebbe per esempio /qualitative_datasets/alley
          if(cartella2==".DS_Store"):
              continue
          sottocartella = os.listdir(filename + '/'+ cartella + '/' + cartella2)
          if(len(sottocartella)<2):
            continue
            #sottocartella2 è per esempio /qualitative_datasets/alley/input
          for sottocartella2 in sottocartella:
            if(sottocartella2==".DS_Store"):
              continue
            lista_immagini = os.listdir(filename + '/' + cartella + '/' + cartella2 + '/' + sottocartella2)
            if(sottocartella2=="input"):
              for immagine in lista_immagini:
                if(immagine==".DS_Store"): 
                    continue
                x = appendImmagine(filename + '/' + cartella + '/' + cartella2 + '/' + sottocartella2 + '/' + immagine, x)
            else:
                for immagine in lista_immagini:
                    if(immagine==".DS_Store"): 
                        continue
                    y = appendImmagine(filename + '/' + cartella + '/' + cartella2 + '/' + sottocartella2 + '/' + immagine, y)
    return x, y

def ottieniImmaginiTest(filename,x, y):
    #apro la cartella del dataset
    entries = os.listdir(filename)
    #entries.pop(0)
    #apro ogni sottocartella
    print("LOADING DATASET:")
    for cartella in entries:
        if(cartella=="input"):
            lista_immagini = os.listdir(filename + '/' + cartella)
            for immagine in lista_immagini:
                if(immagine==".DS_Store"): 
                    continue
                x = appendImmagine(filename + '/' + cartella + '/' + immagine, x)
        elif(cartella==".DS_Store"):
          continue
        else:
            lista_immagini = os.listdir(filename + '/' + cartella)
            for immagine in lista_immagini:
                if(immagine==".DS_Store"): 
                    continue
                y = appendImmagine(filename + '/' + cartella + '/' + immagine, y)
    return x, y

In [ ]:
#x, y = ottieniImmagini('/content/DeepVideoDeblurring_Dataset', x , y)
x,y = ottieniImmaginiTest('/content/DeepVideoDeblurring_Dataset/quantitative_datasets/720p_240fps_1',x,y)
#
x = np.array(x)
y = np.array(y)


LOADING DATASET:


In [ ]:
print(np.shape(x),np.shape(y))

(100, 720, 1280, 3) (100, 720, 1280, 3)


In [ ]:
#cercare un modo per stampare meglio queste immagini

import matplotlib.pyplot as plt

n_rows = 2
n_cols = 2


plt.figure(figsize=(n_cols * 28.4, n_rows * 28.6))
for row in range(n_rows):
    for col in range(n_cols):
        index = 2*(n_cols * row + col)
        plt.subplot(n_rows, 2*n_cols, index + 1)
        plt.imshow(x[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.subplot(n_rows, 2*n_cols, index + 2)
        plt.imshow(y[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        #plt.title(class_names[y_train[index][0]]) #mettere come titolo "sfocato" , "definito"
        
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
#normaliziamo l'input tra 0 e 1

x_train = x_train/255
x_test = x_test/255
y_train = y_train/255
y_test = y_test/255

print(np.shape(x_train))
print(np.max(x_train), np.min(y_train))


(80, 720, 1280, 3)
1.0 0.0


In [ ]:
print(np.shape(x_train)[1:])

(720, 1280, 3)


In [ ]:
#definiamo il nostro modello
input_shape = np.shape(x_train)[1:]

model = keras.models.Sequential([
    #encoder
    keras.layers.Conv2D(filters=32, kernel_size=[3,3], padding="same", activation="relu", input_shape=input_shape),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=16, kernel_size=[2,2], activation='relu'),
    keras.layers.MaxPool2D(pool_size=[2,2]),
    #decoder
    keras.layers.Conv2DTranspose(32, (2, 2), strides=2, activation="relu", padding="same"),
    keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same"),
    keras.layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")
    #keras.layers.Dense(10, activation="softmax")
    
    # bisogna aggiustare i parametri in modo che l'output abbia dimensioni uguali a input_shape  ossia (720, 1280, 3)
    
  ])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer='adam',
              metrics=["mse"]) #mse anzichè accuracy
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 720, 1280, 32)     896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 360, 640, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 359, 639, 16)      2064      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 179, 319, 16)     0         
 2D)                                                             
                                                                 
 conv2d_transpose_6 (Conv2DT  (None, 358, 638, 32)     4640      
 ranspose)                                                       
                                                      

In [ ]:
# train the model
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=10,
    batch_size=128,
    verbose = 1,
    #shuffle=True,
    validation_data=(x_test, y_test),
)


In [ ]:
#cambiare accuracy con mse

def plot_loss(history):
  plt.figure(figsize=(10,6))
  plt.plot(history.epoch,history.history['loss'], label='loss')
  plt.plot(history.epoch,history.history['val_loss'],label='val_loss')
  plt.title('loss')
  plt.legend()
  
def plot_accuracy(history):
  plt.figure(figsize=(10,6))
  plt.plot(history.epoch,history.history['accuracy'],label='accuracy')
  plt.plot(history.epoch,history.history['val_accuracy'],label='val_accuracy')
  plt.title('accuracy')
  plt.legend()
  
plot_loss(history)

plot_accuracy(history)

scores = model.evaluate(x_test, y_test, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# mostrare di nuovo le immagini correte dalla rete neurale rispetto a quelle vere